In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### EnPhase Getting Started
- A starter notebook to sort out some of the basic syntax, data nodels, data content, etc. of EnPhase data


In [23]:
import datetime as dtm
import matplotlib.dates as mpd
import pytz
tzutc = pytz.timezone('UTC')
import h5py
#
#import operator
import math
import random
import numpy
import scipy
#import scipy.optimize as spo
from scipy import interpolate
import scipy.constants
import itertools
import sys
#import scipy.optimize as spo
import os
#import operator
#from PIL import Image as ipp
import multiprocessing as mpp
#
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
#import json
#import pickle
#
import requests
#
import geopy.distance
#from geopy.distance import vincenty
#from geopy.distance import great_circle
#
#import shapely.geometry as sgp
os.environ['PROJ_LIB'] = '{}/anaconda3/share/proj'.format(os.getenv('HOME'))
#
from mpl_toolkits.basemap import Basemap as Basemap
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from geographiclib.geodesic import Geodesic as ggp
#
#
import random
import geopy
import json

In [20]:
enphase_data_file='enphase_ids.csv'
enphase_data={}
with open(enphase_data_file, 'r') as fin:
    for rw in fin:
        if rw.startswith('#'):
            continue
        #
        ky, vl = rw.replace('\n', '').split('\t')
        enphase_data[ky] = vl
    #
#
print('enphase_data: ', enphase_data)
#
api_key = enphase_data['api_key']
#
# I don't actually understand what the user_id is. this is a demo-user id...
enphase_user_id = '4d7a45774e6a41320a'
        

enphase_data:  {'api_key': '9df0b8fc269146793c862d8e241aa220'}


## First demo query:

### Summary:
- I confess that at this time, I don't quite get the user_id parameter. I don't see a user_id code for my ID; my ID does not work, but I can use the demo-user and my api-key.
- This query will give a "summary" record for system_id=67 (unless we change that variable later)
- Note that we're being a little sloppy about the query string; the more proper way to do this is probably to use some sort of "encode()" function to build the QS, otherwise, we can make a lot of silly mistakes.


In [21]:
# fetch some data:
# TODO: might need to use "encode" and other functions to build url and/or query string.
#. but also, be careful, because there are multiple standards for q-string delimitation.
#. i think i recently ran into this with the USGS ComCat API.
#
sys_id=67
dataset='summary'
#response = requests.get('https://api.enphaseenergy.com/api/v2/systems/67/summary?key={}'.format(enphase_data['api_key']))
url = 'https://api.enphaseenergy.com/api/v2/systems/{}/{}?key={}&user_id={}'.format(
    sys_id, dataset, enphase_data['api_key'], enphase_user_id)
print('** url: ', url)

response = requests.get(url)
#
print('** response: ', response)





** url:  https://api.enphaseenergy.com/api/v2/systems/67/summary?key=9df0b8fc269146793c862d8e241aa220&user_id=4d7a45774e6a41320a
** response:  <Response [200]>


In [25]:
#
print('*** response return code: ', response.status_code)

data = response.content.decode()
print('***\n', data)

*** response return code:  200
***
 {"system_id":67,"modules":31,"size_w":7690,"current_power":24,"energy_today":90,"energy_lifetime":112069333,"summary_date":"2019-12-04","source":"microinverters","status":"normal","operational_at":1201362300,"last_report_at":1575479729,"last_interval_end_at":1575478848}


In [30]:
# make the request by passing parameters for the q-string.
#
core_req_prams = {'key': api_key, 'user_id': enphase_user_id}
url2 = 'https://api.enphaseenergy.com/api/v2/systems/{}/{}'.format(sys_id, dataset)
#
response2 = requests.get(url2, params=core_req_prams,)
#
print('** code: ', response2.status_code)
data = response.content.decode()
#
print('** data: ', data)


** code:  200
** data:  {"system_id":67,"modules":31,"size_w":7690,"current_power":24,"energy_today":90,"energy_lifetime":112069333,"summary_date":"2019-12-04","source":"microinverters","status":"normal","operational_at":1201362300,"last_report_at":1575479729,"last_interval_end_at":1575478848}


## Index Query
- This is really our starting place (except that the first decent demo query we found (above) was for 'summary'
- Do some index queries. Note, this will return multiple results. We can filter if we want
- We will demo how to do multiple queries to select $N>N_{max}$. This is refered to under the scope of "pagination" in the documentation.
- Basically, we'll do a query; note the ID of the final entry, and use that as the start point for the next query.


In [39]:
url_index = 'https://api.enphaseenergy.com/api/v2/systems'
#
# some prams are {next, limit, status, or status[] for multiple status values}
index_prams = {'limit':1000}
index_prams.update(core_req_prams)
#
response_index = requests.get(url_index, params=index_prams)
#
print('*** code: ', response_index.status_code)
#

#data_b = response_index.content

data_json = json.loads(response_index.content.decode())

print('keys: ', data_json.keys()

print('next ID: ', data_json['next'])
print('len(systems): ', len(data_json['systems']))

next ID:  4e4441344d7a41340a
len(systems):  1000


In [55]:
# This generally works, but not all entries have identical keys. It looks like some, but not all, entries
#. have a key:val called "other_references"
#
#
kys0 = data_json['systems'][0].keys()
n_errors = 0
unique_keys = []
for k,rw in enumerate(data_json['systems'][1:]):
    #print('** rw[{}]: {}'.format(k,rw) )
    #
    for ky in rw.keys():
        if not ky in unique_keys:
            unique_keys += [ky]
    #
    #
    kys = rw.keys()
    if not list(kys)==list(kys0):
        #print('***  error: keys[{},{}] do not match!!!'.format(k, k+1))
        #print('* * ', kys0)
        #print('* * ', kys)
        n_errors += 1
    kys0=kys
    #
    #if k>15: break
#
print('** n_key_errors: ', n_errors)
print('** unique keys: ', unique_keys)

** n_key_errors:  390
** unique keys:  ['system_id', 'system_name', 'system_public_name', 'status', 'timezone', 'country', 'state', 'city', 'postal_code', 'connection_type', 'meta', 'other_references']


In [48]:
print('*** ', data_json['systems'][0])

***  {'system_id': 67, 'system_name': 'Eich Residence', 'system_public_name': 'Eich Residence', 'status': 'normal', 'timezone': 'America/Los_Angeles', 'country': 'US', 'state': 'CA', 'city': 'Sebastopol', 'postal_code': '95472', 'other_references': ['Solarfox'], 'connection_type': 'wifi', 'meta': {'status': 'normal', 'last_report_at': 1575481783, 'last_energy_at': 1575481069, 'operational_at': 1201362300}}


### Something more operational...
1. Determine which column data we want to collect and handle cases where those data do not exist
2. Note that we basically have to decide which columns to use up front. We can write scripts to do this dynamically, but in practice we'll usually spin through the data to determine/guess a unique and/or desired set of columnns
2. How do we concatenate with multiple queries?

This nested-dict type data structure is really nice for row-by-row work, and being totally unstructured can make it easy to produce and maintain, but a bit messy and slow to work with. But nothing we can't handle.

Let's get 2000 records.

### What we find:
- We correctly show how to do an index query 1000 records (or any n) at a time
- We (almost) correctly tansform the JSON dict-like structure to a structured list
- We discover that the 'meta' column is a nested dict-like structure, so we'll need to look at that more closely. What questions do we want to ask? Do we need to expand those out?
- Operatioally, do we need to just pull this all back to a similar data structure?


In [62]:
n_records = 2000
n_batch=1000
n_steps = int(numpy.ceil(n_records/n_batch))
#
print('query steps:\nn_records: {}, n_batch: {}, n_steps: {}'.format(n_records, n_batch, n_steps))
#
# we can use any number of data containers. We should probably use a rec-array, though most people would
#. likley opt for a PANDAS data frame. right now, I don't want to sort out the string-lengths for rec-array
#. and I hate PANDAS (though I confess it is probably a good choice to get started here). For nowk
#. let's just make a cols-header and lump the data into a list-type. Note that we can pre-construct
#
delim = chr(9)    # tab
cols = unique_keys
data = []
index_prams = {'limit':1000}
index_prams.update(core_req_prams)
next_key=''
#
for step in range(n_steps):
    #
    my_prams = index_prams.copy()
    if not next_key=='':
        my_prams['next']=next_key
    print('** qs_prams: ', my_prams)
    #
    response_index = requests.get(url_index, params=my_prams)
    #
    print('*** code: ', response_index.status_code)
    #
    # TODO: in real life, we'd have some error handling here. we would evaluate the response_code
    #. to determine whether or not the query was successful.
    #
    data_json = json.loads(response_index.content.decode())
    next_key = data_json['next']
    #
    # now, spin through the systems:
    for sys_rw in data_json['systems']:
        #
        # could also do this with by fetching key,vals and sorting by cols or something, but
        #. this is pretty bullet-proof:
        data += [[sys_rw.get(ky, None) for ky in cols]]
    #
    print('keys: ', data_json.keys())
    print('next ID: ', data_json['next'])
    print('len(systems): ', len(data_json['systems']))
    

query steps:
n_records: 2000, n_batch: 1000, n_steps: 2
** qs_prams:  {'limit': 1000, 'key': '9df0b8fc269146793c862d8e241aa220', 'user_id': '4d7a45774e6a41320a'}
*** code:  200
keys:  dict_keys(['systems', 'next'])
next ID:  4e4441344d7a41340a
len(systems):  1000
** qs_prams:  {'limit': 1000, 'key': '9df0b8fc269146793c862d8e241aa220', 'user_id': '4d7a45774e6a41320a', 'next': '4e4441344d7a41340a'}
*** code:  200
keys:  dict_keys(['systems', 'next'])
next ID:  4e6a4d344f5459790a
len(systems):  1000


In [63]:
# let's have a quick look at the data:
#
print('len(data): ', len(data))
#

print('cols: {}\n'.format(cols))
for rw in data[0:10]:
    print('** ', rw)

len(data):  2000
